# Blood Pressure Assessment Over Time

This notebook generates bar charts showing the frequency of each blood pressure assessment category per month and year.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Limits for high and low blood pressure readings
systolic_threshold = 140
diastolic_threshold = 90

# Threshold used to identify gaps in the data and segment it for plotting
gap_threshold = 7

In [ ]:
%run api.ipynb
%run config.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_daily_blood_pressure_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# Add a "month" column
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()
df["year"] = df["date"].dt.to_period("Y").dt.to_timestamp()

# Group by month/year and assessment
monthly_grouped = df.groupby(["month", "assessment"]).size().reset_index(name="count")
yearly_grouped = df.groupby(["year", "assessment"]).size().reset_index(name="count")

# Pivot for plotting
monthly_pivot = monthly_grouped.pivot(index="month", columns="assessment", values="count").fillna(0)
yearly_pivot = yearly_grouped.pivot(index="year", columns="assessment", values="count").fillna(0)

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
monthly_pivot.plot(kind="bar", stacked=True, ax=ax, colormap="tab20")

# Title and axis labels
ax.set_title("Monthly Frequency of Blood Pressure Assessment Categories")
ax.set_xlabel("Month")
ax.set_ylabel("Count")

# Customize x-tick labels
xticks = ax.get_xticks()
xtick_labels = monthly_pivot.index.strftime("%Y-%m")
ax.set_xticks(xticks[::2])
ax.set_xticklabels(xtick_labels[::2], rotation=90)

# Move legend below the plot
ax.legend(
    title="Assessment",
    bbox_to_anchor=(0.5, -0.25),
    loc="upper center",
    ncol=3,
    frameon=False
)

# Layout adjustments
plt.tight_layout()

# Export to PNG
export_file_name = "Assessment-Frequency-Over-Time"
if chart_export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Monthly.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if chart_export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Monthly.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the chart
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
yearly_pivot.plot(kind="bar", stacked=True, ax=ax, colormap="tab20")

# Title and axis labels
ax.set_title("Annual Frequency of Blood Pressure Assessment Categories")
ax.set_xlabel("Year")
ax.set_ylabel("Count")

# Customize x-tick labels
xticks = ax.get_xticks()
xtick_labels = yearly_pivot.index.strftime("%Y")
ax.set_xticks(xticks)
ax.set_xticklabels(xtick_labels, rotation=90)

# Move legend below the plot
ax.legend(
    title="Assessment",
    bbox_to_anchor=(0.5, -0.25),
    loc="upper center",
    ncol=3,
    frameon=False
)

# Layout adjustments
plt.tight_layout()

# Export to PNG
export_file_name = "Assessment-Frequency-Over-Time"
if chart_export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Yearly.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if chart_export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Yearly.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the chart
plt.show()